In [1]:
import json
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Load data
data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\search_20241106-223705_sodium+ion+battery+anode-sodium+ion+battery+cathode-sodium+ion+battery+electrode_annotated_rephrased.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Prepare data
texts = []
labels_raw = []
for i in data:
    texts.append(i["text"])
    labels_raw.append(i["label_int"])

labels = []
for i in labels_raw:
    if i == 0:
        labels.append(1)
    elif i == 1:
        labels.append(0)

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)

# Create datasets
train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})
val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})
raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
print(len(texts))

400


In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 80
    })
})


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers.trainer_callback import TrainerCallback
import evaluate
import numpy as np
from tqdm.auto import tqdm

# Set seed for reproducibility
torch.manual_seed(42)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the tokenizer and model
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assuming raw_datasets is a DatasetDict with "train" and "validation" splits
# and each example has "text" and "labels" fields

# Function to tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization to the datasets
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Load the model
num_labels = len(set(raw_datasets["train"]["labels"]))
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model = model.to(device)  # Move model to GPU

# Load evaluation metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Create a proper callback by inheriting from TrainerCallback
class EpochReportCallback(TrainerCallback):
    def __init__(self):
        self.epoch = 0
    
    def on_epoch_end(self, args, state, control, **kwargs):
        self.epoch += 1
        
        # Get the most recent training loss
        if len(state.log_history) > 0:
            # Find the most recent training loss entry
            train_losses = [log for log in state.log_history if "loss" in log and "eval" not in log]
            if train_losses:
                train_loss = train_losses[-1]["loss"]
                print(f"\nEpoch {self.epoch}/{args.num_train_epochs}")
                print(f"Training Loss: {train_loss:.4f}")
            
            # Find the most recent evaluation entries
            eval_entries = [log for log in state.log_history if "eval_loss" in log]
            if eval_entries:
                eval_loss = eval_entries[-1]["eval_loss"]
                eval_accuracy = eval_entries[-1]["eval_accuracy"]
                print(f"Evaluation Loss: {eval_loss:.4f}")
                print(f"Evaluation Accuracy: {eval_accuracy:.4f}")
        
        return control

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # Add CUDA specific arguments
    fp16=True,  # Use mixed precision training
    no_cuda=False,  # Use CUDA
    report_to="none",  # Disable default logging to avoid clutter
)

# Create the Trainer with an instance of the callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EpochReportCallback()],  # Create an instance of the callback class
)

# Train the model
print("Starting training...")
trainer.train()

Using device: cuda


Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Starting training...


  0%|          | 0/400 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.2961278557777405, 'eval_accuracy': 0.9625, 'eval_runtime': 0.279, 'eval_samples_per_second': 286.782, 'eval_steps_per_second': 35.848, 'epoch': 9.0}
{'loss': 0.0001, 'grad_norm': 0.0027870074845850468, 'learning_rate': 3.6700000000000004e-05, 'epoch': 9.25}
{'loss': 0.0, 'grad_norm': 0.003563058329746127, 'learning_rate': 3.77e-05, 'epoch': 9.5}
{'loss': 0.0, 'grad_norm': 0.0012248293496668339, 'learning_rate': 3.8700000000000006e-05, 'epoch': 9.75}
{'loss': 0.0, 'grad_norm': 0.001021943404339254, 'learning_rate': 3.97e-05, 'epoch': 10.0}

Epoch 10/10
Training Loss: 0.0000
Evaluation Loss: 0.2961
Evaluation Accuracy: 0.9625


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.31184953451156616, 'eval_accuracy': 0.9625, 'eval_runtime': 0.2711, 'eval_samples_per_second': 295.141, 'eval_steps_per_second': 36.893, 'epoch': 10.0}
{'train_runtime': 53.921, 'train_samples_per_second': 59.346, 'train_steps_per_second': 7.418, 'train_loss': 0.2022728704288602, 'epoch': 10.0}


TrainOutput(global_step=400, training_loss=0.2022728704288602, metrics={'train_runtime': 53.921, 'train_samples_per_second': 59.346, 'train_steps_per_second': 7.418, 'total_flos': 841955377152000.0, 'train_loss': 0.2022728704288602, 'epoch': 10.0})

In [5]:
# Save the fine-tuned model
model.save_pretrained("./batterybert-cased-abstract_finetuned_400_2")
tokenizer.save_pretrained("./batterybert-cased-abstract_finetuned_400_2")

('./batterybert-cased-abstract_finetuned_400_2\\tokenizer_config.json',
 './batterybert-cased-abstract_finetuned_400_2\\special_tokens_map.json',
 './batterybert-cased-abstract_finetuned_400_2\\vocab.txt',
 './batterybert-cased-abstract_finetuned_400_2\\added_tokens.json',
 './batterybert-cased-abstract_finetuned_400_2\\tokenizer.json')

In [6]:
# For inference
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to GPU
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.cpu().numpy()  # Move predictions back to CPU for numpy conversion

In [13]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_1.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = predict(val_data["text"][i])
    if val_label[0] == 0:
        label_bert.append(1)
        # score_bert.append(val_label[0]["score"])
    elif val_label[0] == 1:
        label_bert.append(0)
        # score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert
print(len(label_bert))

100


In [14]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_fp = 0
openai_fn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1

print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
7	0	1	0
21	0	1	0
34	0	1	0
36	0	1	0
53	0	1	0
72	1	1	0
79	0	1	0
80	0	1	0
83	0	0	1
99	0	1	0
OpenAI False Positive: 8
OpenAI False Negative: 0
Bert False Positive: 1
Bert False Negative: 1
